In [1]:
from enum import Enum
class Lang(Enum):
    EN = 1
    RU = 2

# choose language (affect movie dataset)
ENGINE = Lang.RU

In [2]:
# solve problem with mystem package in google colab
!if [["$OSTYPE" == "linux-gnu"]]; then wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz && tar -xvf mystem-3.0-linux3.1-64bit.tar.gz && cp mystem /bin; fi

fish: Missing end to balance this if statement
if [["$OSTYPE" == "linux-gnu"]]; then wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz && tar -xvf mystem-3.0-linux3.1-64bit.tar.gz && cp mystem /bin; fi
^



In [3]:
import spacy
import pandas as pd
from pymystem3 import Mystem

def abs_filepath(file):
    if 'google.colab' in str(get_ipython()):
        return "https://raw.githubusercontent.com/madmaxeatfax/fellini/master/datasets/" + file
    return "./datasets/" + file

def get_stopwords(lang):
    f = open(abs_filepath(f"stopwords_{lang}.txt"))
    stopwords = set()
    for word in f:
        stopwords.add(word.rstrip())
    f.close()
    return stopwords


class EnLemmaTokenizer:
    def  __init__(self):
        self.nlp = spacy.load("en")
        self.stopwords = get_stopwords("en")
    def __call__(self, text):
        return [
            token.lemma_ for token in self.nlp(text)
            if token.lemma_ not in self.stopwords 
            and token.lemma_[0].isalpha()
        ]

class RuLemmaTokenizer:
    def  __init__(self):
        self.lemma = Mystem().lemmatize
        self.stopwords = get_stopwords("ru")
    def __call__(self, text):
        return [
            token for token in self.lemma(text)
            if token not in self.stopwords and token[0].isalpha()
        ]

lemma_tokenizer = EnLemmaTokenizer() if ENGINE == Lang.EN else RuLemmaTokenizer()

kp_dataset, imdb_dataset = abs_filepath("kp250.csv"), abs_filepath("imdb250.csv")
bunch = pd.read_csv(imdb_dataset if ENGINE == Lang.EN else kp_dataset)
bunch.fillna("", inplace=True)

In [4]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(max_features=50_000, tokenizer=lemma_tokenizer)
data = \
    bunch.Title + " " + bunch.Crew + " " + bunch.Plot + " " + \
    bunch.Tags + " " + bunch.Country + " " + bunch.Reviews

features = vec.fit_transform(data)

words = vec.get_feature_names()
print(f'Dictionary size = {len(words)}')

Dictionary size = 33448
CPU times: user 7.45 s, sys: 397 ms, total: 7.85 s
Wall time: 1min 1s


In [5]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=5, metric='cosine')
knn.fit(features)

NearestNeighbors(metric='cosine')

In [6]:
%%time

# write your search query
QUERY = "фильм в котором бухгалтер убил свою жену и отправился в тюрьму"
query_vector = vec.transform([QUERY])
print(f'Query = \'{QUERY}\'\nTokens =  {lemma_tokenizer(QUERY)}\n')

distances, neighbors = knn.kneighbors(query_vector, return_distance=True)
    
for dist, neighbor_idx in zip(distances[0], neighbors[0]):
    print(*[
        bunch.Title[neighbor_idx],
        f'Distance = {dist}  Neighbor idx = {neighbor_idx}',
        bunch.Plot[neighbor_idx][:200], 
        bunch.Crew[neighbor_idx],
        "-"*200
    ], sep="\n")


Query = 'фильм в котором бухгалтер убил свою жену и отправился в тюрьму'
Tokens =  ['бухгалтер', 'убивать', 'свой', 'жена', 'отправляться', 'тюрьма']

Побег из Шоушенка 
Distance = 0.8320742839978136  Neighbor idx = 0
Бухгалтер Энди Дюфрейн обвинён в убийстве собственной жены и её любовника. Оказавшись в тюрьме под названием Шоушенк, он сталкивается с жестокостью и беззаконием, царящими по обе стороны решётки. Кажд
 Фрэнк Дарабонт,  Фрэнк Дарабонт,  Стивен Кинг, Тим Роббинс, Морган Фриман, Боб Гантон, Уильям Сэдлер, Клэнси Браун
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Зеленая миля 
Distance = 0.8579762353369685  Neighbor idx = 1
Пол Эджкомб — начальник блока смертников в тюрьме «Холодная гора», каждый из узников которого однажды проходит «зеленую милю» по пути к месту казни. Пол повидал много заключённых и надзирателей за вре
 